# Tutorial about images

In the context of SMLM you also have to deal with pixelated images. For instance, raw data is recorded as movies from which localization data is generated. Or widefield microscopy images are recorded together with SMLM data.

To deal with image data, various image processing libraries exist. Locan makes use of a simple image class that serves as container and adapter class for any image class from a third-party library.

In [ ]:
import tempfile
from dataclasses import dataclass
from pathlib import Path

%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt
import httpx
import skimage as ski

import locan as lc
from locan.data.images import Image

In [ ]:
lc.show_versions(system=False, dependencies=False, verbose=False)

In [ ]:
directory = Path(tempfile.mkdtemp())

## Some image

Let's get some image data:

In [ ]:
url = "https://raw.githubusercontent.com/super-resolution/Locan/main/docs/_static/logo_plus.png"

In [ ]:
response = httpx.get(url)
print("Response is ok: ", response.is_success)

In [ ]:
file_path = directory / Path(url).name

with open(file_path, 'wb') as file:
    for chunk in response.iter_bytes(chunk_size=128):
        file.write(chunk)
        
file_path

In [ ]:
logo = ski.io.imread(file_path)
type(logo), logo.shape

In [ ]:
plt.imshow(logo);

## The Image class

In this example, the image is a plain numpy array. However, the image can be a more complex class with a specific interface, depending on the image library used.

Locan provides an adapter class to wrap any image class. The adapter class `locan.Image` provides a minimal standard interface.

In [ ]:
Image?

Objects should be instantiated through an appropriate constructor method.

In [ ]:
[method for method in dir(Image) if method.startswith("from")]

### Image attributes

Upon instantiation `locan.Image` wraps the original image class and provides an array as `image.data`.

Any attribute request is forwarded first to image.data or - if not available - to the original images class referenced through image._image.

In [ ]:
image = Image.from_numpy(array=logo, is_rgb=True)

In [ ]:
image.is_rgb

In [ ]:
image.shape

In [ ]:
image.size

In [ ]:
image.dtype

In [ ]:
image.ndim

### The Image data attribute

Typically, image.data should present the image as an array object that can be used with the array API standard.

If this is not the case a numpy.NDArray is provided.

In [ ]:
image = Image.from_array(array=[[1, 2, 3], [1, 2, 3]])
type(image.data), image.dtype

In [ ]:
image = Image.from_array(array=logo, is_rgb=True)
type(image.data), image.dtype

In [ ]:
plt.imshow(image.data);

You might want to use methods according to the array API standard also with other libraries than np.

In [ ]:
np.sum(image.data)

Alternatively, use a different method implementation from array_api_compat:

In [ ]:
try:
    from array_api_compat import array_namespace
    xp = array_namespace(image.data)
    logo_as_float = xp.astype(image.data, np.float32)
    print(type(logo_as_float), logo_as_float.dtype)
except ImportError as e:
    print("Exception:", e)

### The Image bins attribute

To keep pixel coordinates with the image use the bins attribute:

In [ ]:
image = Image.from_array(array=logo[:, :, 0], is_rgb=False)
image.shape

In [ ]:
image.bins = lc.Bins(n_bins=image.shape, bin_range=[[10, 20],[100, 200]])
[edges[0:3] for edges in image.bins.bin_edges]

In [ ]:
image.bins

### The Image meta attribute

Metadata that relates to some localization data should be kept under the meta attribute:

In [ ]:
image = Image.from_array(array=logo[:, :, 0], is_rgb=False, meta={"comment": "Free text can go here..."})
image.meta

Additional metadata should be added to the meta.map attribute or in a separate attribute:

In [ ]:
exif = {"Model": "Some Camera", "Information": "details"}
image.exif = exif
image.exif

In [ ]:
image.meta.map.update(exif)
image.meta

## Modify the Image class

To create an Image object from any other library object
modify the initialization of self.data and other attributes accordingly. 
    
Here is a contrived example:

In [ ]:
@dataclass
class ThirdPartyImage:
    img = [[1, 2, 3], [1, 2, 3],]
    is_rgb = False
    extra_metadata = {"Information": "details"}

third_party_image = ThirdPartyImage()
third_party_image.img, third_party_image.is_rgb, third_party_image.extra_metadata

In [ ]:
class MyImage(Image):

    @classmethod
    def from_third_party(cls, image, meta=None):
        new_image = cls(
            image=image,
            data=np.asarray(image.img),
            is_rgb=image.is_rgb,
            meta=meta
        )
        new_image.meta.map.update(image.extra_metadata)
        return new_image

In [ ]:
image = MyImage.from_third_party(image=third_party_image)
image.meta

In [ ]:
image.data